In [ ]:
import os
import pandas as pd
#import polars as pl
from pathlib import Path
from os import listdir

## Flere oppgaver
Hent datasettet fra:

https://pvdata.nist.gov/

Data-beskrivelse finnes her:

https://www.nist.gov/system/files/documents/2017/10/04/datadictionary_supplementalcontent.pdf

Hent zip-fil for ground (bulk download) for et år. Pakk ut.

1. Skriv prosedyre som leser inn disse som dataframe, parser datoer riktig, konkatenerer de, og lagrer som en stor .parquet.
2. Finn gjennomsnittlig døgnproduksjon pr. string (ShuntEtot_kWh_Avg(i))
3. Bruk df.plot() til å plotte døgnproduksjon for de ulike string combinerne gjennom året.

## Oppgave 1: Under følger en del hint for å lese inn og skrive parquet
- Bruk os.listdir(mappe_string) 
- Du trenger antakelig skrive noen nøstede for-løkker for å hente ut alle filnavnene.. 
- Les inn dataframe med pd.read_csv() og parse opp datoen.. se pandas-introen for eksempel. 
- Lag en liste df_list = [] hvor du putter alle dfene du har parsa.
- df = pd.concat(df_list) 
- Sjekk: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
- deretter kan du gjøre df.to_parquet("fil.parquet")

In [ ]:
p = Path("/home/magbak/repos/DMBA_2023/onemin-Ground-2015")
out = Path("/home/magbak/repos/DMBA_2023/")
keep_cols = [
 'TIMESTAMP', 
]
kw_cols = [f'ShuntPDC_kW_Avg_{i}' for i in range(1,7+1)]
keep_cols += kw_cols

In [ ]:
years = listdir(p)
dfs = []
for y in years:
    months = listdir(p / y)
    
    for m in months:
        days = listdir(p / y / m)
        for d in days:
            
            f = p/y/m/d            
            df = pd.read_csv(f, usecols=keep_cols)
            df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"])
            dfs.append(df)
        
df = pd.concat(dfs)
df

In [ ]:
parquet_file = out/"solar.parquet"
df.to_parquet(parquet_file)

In [ ]:
df = pd.read_parquet(parquet_file)

## Oppgave 2: Under er noen hint for å beregne gjennomsnittlig døgnproduksjon
- Du kan bruke list-comprehension til å lage en rekke kolonnenavn:
    `[f"ShuntEtot_kWh_Avg({i})" for i in range(1,7+1)]`
- Du må lage en hjelpekolonne som har bare datoen:
    `df["date"] = df["timestamp"].dt.date`
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.date.html
- Etter at du har funnet summen av produksjonen over hvert døgn (en groupby og agg) så kan du ta groupby år og så mean av dette for å finne gjennomsnittlig døgnproduksjon.

In [ ]:
df["date"] = df["TIMESTAMP"].dt.date
df["month"] = df["TIMESTAMP"].dt.month
df["year"] = df["TIMESTAMP"].dt.year
df

In [ ]:
kwh_cols = [f'ShuntPDC_kWh_tot_{i}' for i in range(1,7+1)]

In [ ]:
for (kw, kwh) in zip(kw_cols, kwh_cols):
    df[kwh] = df[kw]/6

In [ ]:
dfgr = df[["year", "month", "date"]+kwh_cols].groupby(["year", "month", "date"]).sum()

In [ ]:
dfgr

In [ ]:
dfgr = dfgr.reset_index(drop=False)
dfgr

In [ ]:
dfgr[["year"] + kwh_cols].groupby("year").mean()

## Oppgave 3: Hint for plotting:

- y er [f"ShuntEtot_kWh_Avg({i})" for i in range(1,7+1)]
- x er "date"?

Har ikke testet dette ennå, så prøv dere fram med apiet. 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

In [ ]:
dfgr.plot("date", kwh_cols)

In [ ]:
atleast0 = (dfgr[kwh_cols] >= 0)

In [ ]:
dfgr[kwh_cols] = dfgr[kwh_cols].where(atleast0, 0)

In [ ]:
dfmo = dfgr[["month"]+ kwh_cols].groupby("month").sum().reset_index(drop=False)

dfmo.plot.bar("month", kwh_cols)

In [ ]:
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
months_dict = {i+1:m for (i,m) in enumerate(months)}
months_dict

In [ ]:
dfmo["month"] = dfmo["month"].replace(months_dict)

In [ ]:
strings = [f"String {i+1}" for (i,_) in enumerate(kwh_cols)]
strings

In [ ]:
string_rename_dict = {k:s for (k,s) in zip(kwh_cols, strings)}
string_rename_dict

In [ ]:
dfmo = dfmo.rename(columns=string_rename_dict)
dfmo

In [ ]:
dfmo.plot.bar("month", strings, xlabel="Month", ylabel="kWh").legend(bbox_to_anchor=(1.0, 1.0))